# 0. 패키지 불러오기

In [1]:
# 패키지 불러오기 -> 한 번만 하면 아마 뒤부터는 안해도 될 듯
!pip install pandas
!pip install numpy
!pip install catboost
!pip install lightgbm
!pip install category_encoders
!pip install optuna
!pip install seaborn
!pip install matplot
!pip install imblearn
!pip install pacmap

In [84]:
# 패키지 import 시키기 이건 해줘야 함
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import math
import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.ensemble import * # ExtraTreesRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold # KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, f1_score, confusion_matrix 
# make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import LabelEncoder, StandardScaler # LabelEncoder, StandardScaler
# from sklearn.svm import * # SVC, SVR
# from sklearn.inspection import *
# from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
# from sklearn.decomposition import * # PCA
# from sklearn.cluster import *
# import sklearn.neighbors._base

# from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, CatBoostRegressor
# from sklearn.cluster import KMeansuc
# from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
import random
from imblearn.over_sampling import SMOTE, ADASYN
from FRUFS import FRUFS
import pacmap
# from missingpy import MissForest

from lightgbm import LGBMClassifier, LGBMRegressor
from datetime import datetime
import calendar
from collections import Counter

import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

In [65]:
# 경로 확인
!ls '/home/studio-lab-user/MYDATA'

'Construction Machine Oil'  'Getting Started.ipynb'   images


In [66]:
# 경로 확인2
!ls '/home/studio-lab-user/MYDATA/Construction Machine Oil/open/'

data_info.xlsx	sample_submission.csv  test.csv  train.csv


In [67]:
# 확인한 경로대로 train 데이터와 test 데이터 불러오기
path = '/home/studio-lab-user/MYDATA/Construction Machine Oil/open/'

Rdata_train = pd.read_csv(path + 'train.csv')
Rdata_test = pd.read_csv(path + 'test.csv')

print(Rdata_train.shape) # (14095, 54) -> 생각보다 데이터 수가 적다.
print(Rdata_train.size)
print(Rdata_test.shape) # (6041, 19)
print(Rdata_test.size)
Rdata_train.head()

(14095, 54)
761130
(6041, 19)
114779


,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


In [68]:
# 데이터 copy 작업
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

In [69]:
# 변수 정하기
# 아마 작업한다면 여기를 신경써야 할 것이다.
# 어떤 변수를 빼고 넣어야 할지 다양하게 시도해 볼 것

train2 = train1.loc[:, ['AL', 'BA', 'Y_LABEL',
                        'ANONYMOUS_1', 'NI', 'CR', 'MN', 'FE', 'CU', 'PQINDEX', 'YEAR']]
test2 = test1.loc[:, ['ANONYMOUS_1', 'NI', 'CR', 'MN', 'FE', 'CU', 'PQINDEX', 'YEAR']] # test 데이터에 ID 변수는 빼줘야 함

print(train2.shape)
print(test2.shape)

(14095, 11)
(6041, 8)


In [70]:
# 범주형 변수를 숫자로 바꾸기
# 이로써 COMPONENT_ARBITRARY -> COMPONENT_ARBITRARY_category로 이름도 바뀌고 데이터도 바뀜
# YEAR -> YEAR_category로 바뀜
le1 = LabelEncoder()
le2 = LabelEncoder()

# train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

# test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

train3 = train2.drop(['YEAR'], axis = 1)
test3 = test2.drop(['YEAR'], axis = 1)

In [71]:
# CatBoost에서는 반드시 범주형 변수가 어떤 것인지를 지정해줘야함
# 따라서 categorical_features 처럼 지정해줄 리스트가 필요
# 사실 그 밑에 numeric 리스트 2개는 필요 없음

categorical_features = ['YEAR_category']

print(train3.shape)
print(test3.shape)

(14095, 11)
(6041, 8)


In [72]:
# train 데이터에서 X_train은 Y_LABEL 빼고, y_train은 Y_LABEL만 넣고
X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

In [73]:
# TRAIN 데이터를 교차검증을 위해 PARTRAIN과 VAL로 나누기
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, 
                                                        y_train, 
                                                        test_size = 0.3, 
                                                        random_state = 39, 
                                                        stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 10)
(4229, 10)
(9866,)
(4229,)


In [74]:
# OPTUNA를 위한 작업

def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1), # -> 0.001에서 1 사이에 랜덤한 연속형 숫자를 지정해준다. learning_rate : 학습률
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000), # -> 100에서 1000까지 랜덤한 정수형 숫자를 지정해준다. n_estimators : 나무의 개수
        "max_depth" : trial.suggest_int("max_depth", 3, 10) # -> 3에서 10까지 랜덤한 정수형 숫자를 지정해준다. max_depth : 나뭇가지의 개수 
  }
    
    model = CatBoostClassifier(**params_cat)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)], # -> eval_set : 모델 적합은 partrain으로 하지만, 평가는 val로 한다.
              early_stopping_rounds = 100, verbose = False, cat_features = categorical_features) # -> early_stopping_rounds : 100번까지도 성능이 안 좋아지면 멈추자.

    cat_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, cat_pred) # roc_auc_score로 두었지만, f1_score, accuracy_score 등 score는 다양하다.
    
    return AUC

In [12]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",   # 클수록 잘 맞춘다는 뜻이므로 maximize로 선택해준다.
    sampler = sampler) 
study.optimize(objective, n_trials = 100) # 100번을 돌려서 가장 좋은 것을 찾아낸다.

[I 2022-12-09 01:20:46,736] A new study created in memory with name: cat_parameter_opt
[I 2022-12-09 01:20:54,214] Trial 0 finished with value: 0.7667590743854684 and parameters: {'learning_rate': 0.04371872304807245, 'n_estimators': 818, 'max_depth': 9}. Best is trial 0 with value: 0.7667590743854684.
[I 2022-12-09 01:21:04,517] Trial 1 finished with value: 0.7570637835267428 and parameters: {'learning_rate': 0.002323537042351288, 'n_estimators': 642, 'max_depth': 7}. Best is trial 0 with value: 0.7667590743854684.
[I 2022-12-09 01:21:13,714] Trial 2 finished with value: 0.7678855843958668 and parameters: {'learning_rate': 0.024644795423723085, 'n_estimators': 524, 'max_depth': 8}. Best is trial 2 with value: 0.7678855843958668.
[I 2022-12-09 01:21:17,326] Trial 3 finished with value: 0.7578024962258691 and parameters: {'learning_rate': 0.5984000779343428, 'n_estimators': 834, 'max_depth': 10}. Best is trial 2 with value: 0.7678855843958668.
[I 2022-12-09 01:21:20,355] Trial 4 finishe

In [75]:
# 가장 좋았던 score 값과 그 때의 초모수를 알려준다.
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)
# 가장 좋았던 모델 'learning_rate': 0.03142344166841527, 'n_estimators': 513, 'max_depth': 6
# 현재 모델 'learning_rate': 0.06550346273535652, 'n_estimators': 340, 'max_depth': 9 -> 0.7695291574879615

Best Score : 0.08653234054058978
Best trial : {'learning_rate': 0.013666536260931852, 'n_estimators': 953, 'max_depth': 9}


In [76]:
# 교차 검증을 통한 모델 적합
# training model for CV #5 가 뜨면 끝난 것

n_fold = 5 # 데이터를 5 등분해서 교차검증을 실시
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39) # 5등분 하되, y의 비율을 적절히 맞춰서 나눈다. -> y의 개수가 조금밖에 없으므로
 
cat_val = np.zeros((X_train.shape[0], 2))
cat_partrain = np.zeros((X_partrain.shape[0], 2))

print(cat_val.shape)
print(cat_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = 0.06550346273535652,     # 초모수 꼭 바꿔줄 것
        n_estimators = 340,                     # 초모수 꼭 바꿔줄 것
        max_depth = 9)                          # 초모수 꼭 바꿔줄 것

    optuna_cat.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False)

    cat_val[i_val, :] = optuna_cat.predict_proba(X_train.loc[i_val, :])
    cat_partrain += optuna_cat.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [77]:
# 이건 그냥 참고용
# 이걸 통해서 내 모형이 얼마나 맞췄는지를 그냥 볼 수 있다.
# TP : 정상이라 예측했는데 실제로 정상인 것
# FP : 정상이라 예측했는데 실제로 불량인 것
# FN : 불량이라 예측했는데 실제로 정상인 것
# TN : 불량이라 예측했는데 실제로 불량인 것
# 아마 해보면 FP의 비율이 클 것이다. -> 정상 속에 숨어 있는 불량을 뽑아내야 한다.

scores = []
TP = []
FP = []
FN = []
TN = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = cat_val[:, 1]
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)
    TP.append(confusion_matrix(y_train, pred)[0][0])
    FN.append(confusion_matrix(y_train, pred)[0][1])
    FP.append(confusion_matrix(y_train, pred)[1][0])
    TN.append(confusion_matrix(y_train, pred)[1][1])
    

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
temp3 = pd.DataFrame(TP, columns = ['TP'])
temp4 = pd.DataFrame(FP, columns = ['FP'])
temp5 = pd.DataFrame(FN, columns = ['FN'])
temp6 = pd.DataFrame(TN, columns = ['TN'])
scores = pd.concat([temp1, temp2, temp3, temp4, temp5, temp6], axis = 1)
scores


,threshold,score,TP,FP,FN,TN
0,0.00,0.157275,0,0,12892,1203
1,0.02,0.270325,7287,139,5605,1064
2,0.04,0.365877,9852,253,3040,950
3,0.06,0.438913,11022,339,1870,864
4,0.08,0.493683,11651,402,1241,801
5,0.10,0.538787,12023,439,869,764
6,0.12,0.572965,12237,457,655,746
7,0.14,0.591526,12383,484,509,719
8,0.16,0.607266,12485,501,407,702
9,0.18,0.623598,12561,508,331,695


In [78]:
# 예측한 불량률을 다시 TRAIN 데이터에 넣어준다.
# 예측한 불량률이 STUDENT MODEL에서 Y가 될 것이다.

print(cat_val.shape)
print(cat_partrain.shape)

X_train['model1_prob'] = cat_val[:, 1]
print(X_train.shape)

(14095, 2)
(9866, 2)
(14095, 11)


In [118]:
X_train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN']]
y_train2 = cat_val[:, 1]
print(X_train2.shape)
print(y_train2.shape)

X_test = test1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN']]
print(X_test.shape)

(14095, 18)
(14095,)
(6041, 18)


In [119]:
# TEST 데이터에 없는 변수들을 빼줘야 한다.
# X_train에서는 만약 AL, BA 말고도 새로운 변수들을 넣었다면 여기서 빼줘야 한다. 18개 변수 제외하고 말이다.
# X_train에서는 꼭 model1_prob도 빼줘야 한다. -> y_train 이므로
# X_train2 = X_train.loc[:, ['ANONYMOUS_1', 'NI', 'CR', 'MN', 'FE', 'CU', 'PQINDEX', 'YEAR_category']]
# y_train2 = X_train['model1_prob']
# print(X_train2.shape)
# print(y_train2.shape)
# print(X_test.shape)

In [120]:
le1 = LabelEncoder()
le2 = LabelEncoder()

X_train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(X_train2['COMPONENT_ARBITRARY'])
X_train2['YEAR_category'] = le2.fit_transform(X_train2['YEAR'])

X_test['COMPONENT_ARBITRARY_category'] = le1.transform(X_test['COMPONENT_ARBITRARY'])
X_test['YEAR_category'] = le2.transform(X_test['YEAR'])

X_train2.drop(['COMPONENT_ARBITRARY'], axis = 1, inplace = True)
X_test.drop(['COMPONENT_ARBITRARY'], axis = 1, inplace = True)

X_train2.drop(['YEAR'], axis = 1, inplace = True)
X_test.drop(['YEAR'], axis = 1, inplace = True)

print(X_train2.shape)
print(X_test.shape)

(14095, 18)
(6041, 18)


In [121]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
temp_train = X_train2.copy()
temp_test = X_test.copy()
X_train2 = sc.fit_transform(X_train2)
X_test = sc.transform(X_test)

print(X_train2.shape)
print(X_test.shape)

(14095, 18)
(6041, 18)


In [122]:
PM = pacmap.PaCMAP(n_components = 18, verbose = False, n_neighbors = None, MN_ratio = 0.5, FP_ratio = 2.0)
X_train2 = PM.fit_transform(np.array(X_train2), init = "pca")
X_test = PM.transform(np.array(X_test), init = "pca", basis = np.array(X_train2))

In [123]:
X_train2 = pd.DataFrame(X_train2, index = temp_train.index, columns = temp_train.columns)
X_test = pd.DataFrame(X_test, index = temp_test.index, columns = temp_test.columns)

In [124]:
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train2, y_train2, test_size = 0.3, random_state = 39)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 18)
(4229, 18)
(9866,)
(4229,)


In [128]:
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10)
  }
    
    model = CatBoostRegressor(**params_cat)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, verbose = False)

    cat_pred = model.predict(X_val)
    MAE = mean_absolute_error(y_val, cat_pred) # 내가 사용한건 MAE 이지만, RMSE 등 다양한 회귀 지표 들이 있다.
    
    return MAE

In [129]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "minimize", # 여기선 minimize 해준다. 오차가 score이므로 작을 수록 좋다.
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-09 07:07:59,211] A new study created in memory with name: cat_parameter_opt
[I 2022-12-09 07:08:07,711] Trial 0 finished with value: 0.09111468339751001 and parameters: {'learning_rate': 0.04371872304807245, 'n_estimators': 818, 'max_depth': 9}. Best is trial 0 with value: 0.09111468339751001.
[I 2022-12-09 07:08:14,935] Trial 1 finished with value: 0.09228462310325888 and parameters: {'learning_rate': 0.002323537042351288, 'n_estimators': 642, 'max_depth': 7}. Best is trial 0 with value: 0.09111468339751001.
[I 2022-12-09 07:08:22,297] Trial 2 finished with value: 0.09118712166772219 and parameters: {'learning_rate': 0.024644795423723085, 'n_estimators': 524, 'max_depth': 8}. Best is trial 0 with value: 0.09111468339751001.
[I 2022-12-09 07:08:27,490] Trial 3 finished with value: 0.09252094354638427 and parameters: {'learning_rate': 0.5984000779343428, 'n_estimators': 834, 'max_depth': 10}. Best is trial 0 with value: 0.09111468339751001.
[I 2022-12-09 07:08:30,987] Trial 4

In [130]:
# 내가 해봤을 때는 여기서 0.08이 아니라 그 이상값 예를 들어서 0.09가 뜨면
# 보통 성능이 좋지 않았다.
# 0.09 이상 뜬다면 Teacher model을 다시 만들어 볼 것 -> 시간 낭비 하지 말고
# 참고로 가장 좋았던 모델은 0.081이 떴다.

print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)
# 8 모델 'learning_rate': 0.11237055673492984, 'n_estimators': 244, 'max_depth': 9 -> 0.08553467733779072
# 18 모델 'learning_rate': 0.013666536260931852, 'n_estimators': 953, 'max_depth': 9 -> 0.08653234054058978
# pca 모델 'learning_rate': 0.07525919980319846, 'n_estimators': 805, 'max_depth': 10 -> 0.09058143935845717

Best Score : 0.09058143935845717
Best trial : {'learning_rate': 0.07525919980319846, 'n_estimators': 805, 'max_depth': 10}


In [44]:
print(X_train2.shape)
print(y_train2.shape)
print(X_test.shape)

(14095, 18)
(14095,)
(6041, 18)


In [45]:
n_fold = 5
cv = KFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train2.shape[0]))
cat_test = np.zeros((X_test.shape[0]))

print(cat_val.shape)
print(cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train2, y_train2), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostRegressor(
        random_state = 39,
        learning_rate = 0.013666536260931852,    # 초모수 꼭꼭 변경해줘라
        n_estimators = 953,                    # 초모수 꼭꼭 변경해줘라
        max_depth = 9)                        # 초모수 꼭꼭 변경해줘라

    optuna_cat.fit(X_train2.loc[i_trn, :], y_train2[i_trn], verbose = False)

    cat_val[i_val] = optuna_cat.predict(X_train2.loc[i_val, :])
    cat_test += optuna_cat.predict(X_test) / n_fold

(14095,)
(6041,)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [46]:
# 역시 참고용

scores = []
TP = []
FP = []
FN = []
TN = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)
    TP.append(confusion_matrix(y_train, pred)[0][0])
    FN.append(confusion_matrix(y_train, pred)[0][1])
    FP.append(confusion_matrix(y_train, pred)[1][0])
    TN.append(confusion_matrix(y_train, pred)[1][1])
    

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
temp3 = pd.DataFrame(TP, columns = ['TP'])
temp4 = pd.DataFrame(FP, columns = ['FP'])
temp5 = pd.DataFrame(FN, columns = ['FN'])
temp6 = pd.DataFrame(TN, columns = ['TN'])
scores = pd.concat([temp1, temp2, temp3, temp4, temp5, temp6], axis = 1)
scores


,threshold,score,TP,FP,FN,TN
0,0.00,0.158488,117,0,12775,1203
1,0.02,0.167114,1109,19,11783,1184
2,0.04,0.184921,3298,103,9594,1100
3,0.06,0.207492,5888,253,7004,950
4,0.08,0.240174,8431,430,4461,773
5,0.10,0.246140,10062,637,2830,566
6,0.12,0.253647,11162,777,1730,426
7,0.14,0.244113,11858,892,1034,311
8,0.16,0.227689,12258,967,634,236
9,0.18,0.197214,12477,1026,415,177


In [52]:
# 이걸 해주고 나서 score 값이 가장 클 때의 threshold 값을 찾아준다.
# 뭐 보통 0.10 ~ 0.20 사이에 나타날 것이다.
# 그렇다고 해서 꼭 저 사이에만 나타나는 건 아니다. -> 하지만 저 사이값이 아니라면 좋지 않을 가능성이 많다.

scores = []
for threshold in range(100) :
    threshold = threshold / 100
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.99, 100), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores.loc[: 50, :]

# 8 모델 -> 0.256393

,threshold,score
0,0.00,0.158488
1,0.01,0.160811
2,0.02,0.167114
3,0.03,0.175642
4,0.04,0.184921
5,0.05,0.196644
6,0.06,0.207492
7,0.07,0.223834
8,0.08,0.240174
9,0.09,0.250230


In [53]:
scores.loc[scores['score'] == scores['score'].max(), :]

# 최고 모델 -> 0.262664
# 18 모델 -> 0.256393
# 8 모델 -> 0.253647

,threshold,score
12,0.12,0.253647


In [47]:
# 그리고 앞에서 구한 threshold 값을 네번째 줄인
# if cat_test[i] >= [여기]
# [여기] 부분에 넣어줘 본다.
# 만약 Counter(answer)을 보았을 때, 1.0 : ???
# 저 물음표 값이 500에 가깝지 않다면 [여기] 부분에서 0.01을 더하든 빼든 계속 조정해주면서
# 500 가까이 되도록 맞추는 것이 좋을 것이다.
# 궁금하면 이 작업 해주지 말고 그냥 [여기]로 해보면 된다.
# 당신이 맞을 수도 있다.
# 하지만 지금까지 해본거로는 500대가 아니면 성능이 안좋았다.

answer = np.zeros(cat_test.shape[0])

for i in range(cat_test.shape[0]) :
  if cat_test[i] >= 0.12 :
    answer[i] = 1

Counter(answer)

Counter({0.0: 5523, 1.0: 518})

In [48]:
# 제출하면 끝이다.

submission_preds = answer
submission = pd.read_csv('/home/studio-lab-user/MYDATA/Construction Machine Oil/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/home/studio-lab-user/MYDATA/Construction Machine Oil/결과/submission_var_10_8_2.csv', index = False)

모델 선택 전까지 해볼 것

1. SMOTE 다른 변수 모두 있을 때 해보기 X
2. ADASYN X
3. MISSFOREST 써서 결측값 채우고 해보기 △  
-> 일단 더 생각해 봐야 겠지만, 솔직히 통계학적으로 보았을 때는 결측 변수는 너무 많이 결측되면 버리는게 맞음
4. 차원 축소 -> PACMAP 등 -> 나름 써볼 수 있을 듯
5. Student 모델 변수 줄이기 -> 모델 선택 되고 나서
6. GBM 사용해보기 X
7. IsorationForest X
8. 표준화 min-max 사용해보기                 -> 모델 선택 되고 나서
9. DBSCAN, HDBSCAN -> 별로
10. TABNET -> 이건 애들보고 해보라고 하기
11. PermutationImportance 로 변수 찾기       -> 모델 선택 되고 나서

Catboost로 선택되고 나서 할 것

1. Teacher, Student 모델 변수 줄이기 -> 매우 중요
2. 표준화 사용해보기 -> 아마 큰 영향을 없을 듯 -> Tree 모형이므로
3. PermutationImportance 로 변수 찾기
4. 특히 Teacher 모델을 적합시키고 나서 Student 모델의 MAE가 0.8로 떨어져야 함 -> 그래야 좋은 모형이 적합됨  
하지만 대부분 0.9 이상이 나옴
5. 통계 방법 적용시키기
6. Stacking Ensemble
7. 분류 지표(ROC_AUC), 회귀 지표(MAE) 바꾸기
8. 로지스틱 분포로 유의한 변수 찾기
9. 과연 test 데이터의 AL를 만들어낼 수 있을까?